# Data Unifying

In [2]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## 1. aler_1996

In [20]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Clean & unify the 'aler_1996' sheet from mult_var_blast.xlsx
1. Read Excel document (sheet='aler_1996')
2. Display basic info & descriptive statistics
3. Rename columns via column_map; apply unit conversions via unit_coef
4. Select the Golden-12 + extra columns
5. Save raw / clean data as Excel; generate a brief Markdown report
"""

# Parameters
INPUT_FILE = Path("data/mult_var_blast.xlsx")   # source workbook
SHEET_NAME = "aler_1996"                        # sheet to read
RAW_OUT = Path("aler_1996_raw.xlsx")            # raw dump
CLEAN_OUT = Path("aler_1996_clean.xlsx")        # unified version
REPORT_MD = Path("aler_1996_report.md")         # markdown summary

# Column mapping: original to unified
column_map = {
    "mine"      : "site",
    "blast"     : "blast_id",
    "L[m]"      : "hole_length_m",
    "R[m]"      : "num_rows",
    "S/B"       : "spacing_over_burden",
    "SxB[m²]"   : "mesh_area_m2",
    "PF[kg/m³]" : "powder_factor",
    "Eb[kg/m³]" : "bottom_energy",
    "Ec[kg/m³]" : "column_energy",
    "De[ms]"    : "delay_ms",
    "Xcr[m]"    : "in_situ_char_size_m",
    "Xcp[m]"    : "muckpile_char_size_m",
    "FI"        : "fragmentation_index",
    "group"     : "group",
}

# Optional unit conversions
unit_coef = {
    # "delay_ms": 0.001. 
    # However, 'delay' is not selected column.
}

# Golden-12 unified feature list
golden12 = [
    "site", "blast_id",
    "spacing_over_burden",        # S/B
    "benchheight_over_burden",    # H/B  (missing here)
    "burden_over_diameter",       # B/D  (missing here)
    "stemming_over_burden",       # T/B  (missing here)
    "powder_factor",
    "youngs_modulus_gpa",         # missing
    "fragment_median_m",          # missing
    "fragmentation_index",        # FI
    "group",
    "mesh_area_m2",
]

# 1. Load the sheet
print(f"Reading {INPUT_FILE}  (sheet='{SHEET_NAME}')")
df_raw = pd.read_excel(INPUT_FILE, sheet_name=SHEET_NAME)
print("Original shape :", df_raw.shape)

# 2. Quick diagnostics
print("\n== dtypes ==")
print(df_raw.dtypes)
print("\n== descriptive statistics (numeric only) ==")
print(df_raw.describe().T)

# 3. Rename & unit conversion
df = df_raw.rename(columns=column_map, errors="ignore")

for col, coef in unit_coef.items():
    if col in df.columns:
        df[col] = df[col] * coef

# 4. Select & save data
# Save the untouched sheet (raw) for archival
df_raw.to_excel(RAW_OUT, index=False)
print(f"Raw Excel saved {RAW_OUT}")

# Existing columns：df["spacing_over_burden"](r) & df["mesh_area_m2"](A)
mask = df["spacing_over_burden"].notna() & df["mesh_area_m2"].notna()
if mask.any():
    r = df.loc[mask, "spacing_over_burden"]   # ratio S/B
    A = df.loc[mask, "mesh_area_m2"]          # mesh area S×B
    df.loc[mask, "burden_m"]  = (A / r) ** 0.5
    df.loc[mask, "spacing_m"] = (A * r) ** 0.5

if "muckpile_char_size_m" in df.columns:
    df["fragment_median_m"] = df["muckpile_char_size_m"]

for col in golden12:
    if col not in df.columns:
        df[col] = pd.NA

df_clean = df[golden12]
df_clean.to_excel(CLEAN_OUT, index=False)
print(f"Clean Excel saved: {CLEAN_OUT}")

# 5. Generate Markdown report
missing_rate = df_clean.isna().mean().rename("missing_ratio")
describe = df_clean.describe(include="all").T
report_tbl = pd.concat([describe, missing_rate], axis=1)

md_lines = [
    "# Aler 1996 - cleaning report",
    "",
    f"* source file : `{INPUT_FILE}`",
    f"* original shape : {df_raw.shape}",
    f"* clean shape    : {df_clean.shape}",
    "",
    "## Missing-rate & descriptive stats (Golden-12)",
    report_tbl.to_markdown(),
    "",
]

REPORT_MD.write_text("\n".join(md_lines), encoding="utf-8")
print(f"Markdown report written -> {REPORT_MD}")

print("Finished!")


Reading data/mult_var_blast.xlsx  (sheet='aler_1996')
Original shape : (33, 17)

== dtypes ==
mine            object
blast           object
L[m]           float64
R[m]             int64
S/B            float64
SxB[m²]        float64
PF[kg/m³]      float64
Eb[kg/m³]      float64
Ec[kg/m³]      float64
De[ms]           int64
Xcr[m]         float64
Xcp[m]         float64
FI             float64
group           object
Unnamed: 14    float64
variable        object
OBS             object
dtype: object

== descriptive statistics (numeric only) ==
             count       mean        std     min      25%      50%      75%  \
L[m]          33.0  11.272727   4.220654  7.5000   7.5000  10.0000  12.5000   
R[m]          33.0   4.181818   2.888063  1.0000   3.0000   3.0000   7.0000   
S/B           33.0   1.200606   0.038319  1.1300   1.1700   1.2000   1.2400   
SxB[m²]       33.0  26.754545  12.828608  7.5000  25.2000  25.2000  42.0000   
PF[kg/m³]     33.0   0.356767   0.075420  0.2165   0.3030   0

## 2. kulatilake_2010

In [21]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Clean & unify the 'kulatilake_2010' sheet from mult_var_blast.xlsx
Same logic as previous one ('aler_1996')
"""


# Parameters
INPUT_FILE  = Path("data/mult_var_blast.xlsx")
SHEET_NAME  = "kulatilake_2010"
RAW_OUT     = Path("kulatilake_2010_raw.xlsx")
CLEAN_OUT   = Path("kulatilake_2010_clean.xlsx")
REPORT_MD   = Path("kulatilake_2010_report.md")

# Column mapping: original to unified
column_map = {
    "site"          : "site",
    "S/B"           : "spacing_over_burden",
    "H/B"           : "benchheight_over_burden",
    "B/D"           : "burden_over_diameter",
    "T/B"           : "stemming_over_burden",
    "PF[kg/m³]"     : "powder_factor",
    "xB[m]"         : "x_burden_pred_m",
    "E[Gpa]"        : "youngs_modulus_gpa",
    "x50[m]"        : "fragment_median_m",
    "FI"            : "fragmentation_index",
    "group"         : "group",
    "ssB[1/m]"      : "specific_surface_burden",   # = 6/B
    "ss50[1/m]"     : "specific_surface_x50",      # = 6/x50
}

unit_coef = {}            

golden12 = [
    "site", "blast_id",               # No blast_id -> NaN
    "spacing_over_burden",
    "benchheight_over_burden",
    "burden_over_diameter",
    "stemming_over_burden",
    "powder_factor",
    "youngs_modulus_gpa",
    "fragment_median_m",
    "fragmentation_index",
    "group",
    "mesh_area_m2",                   # Not exist -> NaN
]

# 1. Load 
print(f"Reading {INPUT_FILE} (sheet='{SHEET_NAME}')")
df_raw = pd.read_excel(INPUT_FILE, sheet_name=SHEET_NAME)
print("Original shape :", df_raw.shape)

# 2. Diagnostics
print("\n== dtypes ==")
print(df_raw.dtypes)
print("\n== descriptive stats (numeric only) ==")
print(df_raw.describe().T)

# 3. Rename & unit convert
df = df_raw.rename(columns=column_map, errors="ignore")
for col, coef in unit_coef.items():
    if col in df.columns:
        df[col] = df[col] * coef

# 3-a.  derive B, S, fragment_median from ssB / ss50
# (1) derive burden & spacing  (ssB = 6 / B)
if {"specific_surface_burden", "spacing_over_burden"}.issubset(df.columns):
    mask = df["specific_surface_burden"].notna() & df["spacing_over_burden"].notna()
    if mask.any():
        B = 6.0 / df.loc[mask, "specific_surface_burden"]
        r = df.loc[mask, "spacing_over_burden"]
        df.loc[mask, "burden_m"]  = B
        df.loc[mask, "spacing_m"] = r * B            # S = (S/B) × B

# (2) derive fragment_median_m if missing and ss50 present
if "fragment_median_m" not in df.columns and "specific_surface_x50" in df.columns:
    df["fragment_median_m"] = 6.0 / df["specific_surface_x50"]

# 4. Save raw & clean
df_raw.to_excel(RAW_OUT, index=False)
print(f"Raw Excel saved: {RAW_OUT}")

# guarantee Golden-12 columns
for col in golden12:
    if col not in df.columns:
        df[col] = pd.NA
df_clean = df[golden12]
df_clean.to_excel(CLEAN_OUT, index=False)
print(f"Clean Excel saved: {CLEAN_OUT}")

# 5. Markdown report
missing_rate = df_clean.isna().mean().rename("missing_ratio")
report_tbl   = pd.concat([df_clean.describe(include="all").T, missing_rate], axis=1)

md_lines = [
    "# kulatilake 2010 - cleaning report",
    "",
    f"* source file : `{INPUT_FILE}`",
    f"* original shape : {df_raw.shape}",
    f"* clean shape    : {df_clean.shape}",
    "",
    "## Missing-rate & descriptive stats (Golden-12)",
    report_tbl.to_markdown(),
    "",
]
REPORT_MD.write_text("\n".join(md_lines), encoding="utf-8")
print(f"Markdown report written -> {REPORT_MD}")

print("Finished!")


Reading data/mult_var_blast.xlsx (sheet='kulatilake_2010')
Original shape : (90, 18)

== dtypes ==
site            object
S/B            float64
H/B            float64
B/D            float64
T/B            float64
PF[kg/m³]      float64
xB[m]          float64
E[Gpa]         float64
x50[m]         float64
FI             float64
group           object
ssB[1/m]       float64
ss50[1/m]      float64
Unnamed: 13    float64
variable        object
OBS            float64
Unnamed: 16    float64
n              float64
dtype: object

== descriptive stats (numeric only) ==
             count       mean        std        min        25%        50%  \
S/B           90.0   1.201778   0.110134   1.000000   1.160000   1.200000   
H/B           90.0   3.458889   1.638696   1.330000   2.400000   3.190000   
B/D           90.0  27.216333   4.798088  17.980000  24.720000  27.270000   
T/B           90.0   1.267556   0.691722   0.500000   0.830000   1.130000   
PF[kg/m³]     90.0   0.536556   0.239227   0.220

## 3. hudaverdi_2010

In [22]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Clean & unify the 'hudaverdi_2010' sheet from mult_var_blast.xlsx
"""

# Parameters
INPUT_FILE = Path("data/mult_var_blast.xlsx")
SHEET_NAME = "hudaverdi_2010"

RAW_OUT   = Path("hudaverdi_2010_raw.xlsx")
CLEAN_OUT = Path("hudaverdi_2010_clean.xlsx")
REPORT_MD = Path("hudaverdi_2010_report.md")

# Column mapping: original to unified
column_map = {
    "site"          : "site",
    # No blast_id -> NaN
    "S/B"           : "spacing_over_burden",
    "H/B"           : "benchheight_over_burden",
    "B/D"           : "burden_over_diameter",
    "T/B"           : "stemming_over_burden",
    "PF[kg/m³]"     : "powder_factor",
    "xB[m]"         : "x_burden_pred_m",
    "E[Gpa]"        : "youngs_modulus_gpa",
    "x50[m]"        : "fragment_median_m",
    "FI"            : "fragmentation_index",
    "group"         : "group",
    "ssB[1/m]"      : "specific_surface_burden",   # 6 / B
    "ss50[1/m]"     : "specific_surface_x50",      # 6 / x50
}

unit_coef = {}     

golden12 = [
    "site", "blast_id",                  
    "spacing_over_burden",
    "benchheight_over_burden",
    "burden_over_diameter",
    "stemming_over_burden",
    "powder_factor",
    "youngs_modulus_gpa",
    "fragment_median_m",
    "fragmentation_index",
    "group",
    "mesh_area_m2",                      
]

# 1. Load
print(f"Reading {INPUT_FILE} (sheet='{SHEET_NAME}')")
df_raw = pd.read_excel(INPUT_FILE, sheet_name=SHEET_NAME)
print("Original shape :", df_raw.shape)

# 2. Diagnostics 
print("\n== dtypes ==")
print(df_raw.dtypes)
print("\n== descriptive stats (numeric only) ==")
print(df_raw.describe().T)

# 3. Rename & unit convert 
df = df_raw.rename(columns=column_map, errors="ignore")
for col, coef in unit_coef.items():
    if col in df.columns:
        df[col] = df[col] * coef

# 3-B.  derive B, S, fragment_median from ssB / ss50 
# (1) derive burden & spacing  (ssB = 6 / B)
if {"specific_surface_burden", "spacing_over_burden"}.issubset(df.columns):
    mask = df["specific_surface_burden"].notna() & df["spacing_over_burden"].notna()
    if mask.any():
        B = 6.0 / df.loc[mask, "specific_surface_burden"]     # B = 6 / ssB
        r = df.loc[mask, "spacing_over_burden"]               # S/B
        df.loc[mask, "burden_m"]  = B
        df.loc[mask, "spacing_m"] = r * B                     # S = (S/B)·B

# (2) derive fragment_median_m if missing and ss50 present
if "fragment_median_m" not in df.columns and "specific_surface_x50" in df.columns:
    df["fragment_median_m"] = 6.0 / df["specific_surface_x50"]

# 4. Save raw & clean 
df_raw.to_excel(RAW_OUT, index=False)
print(f"Raw Excel saved: {RAW_OUT}")

# guarantee Golden-12 columns
for col in golden12:
    if col not in df.columns:
        df[col] = pd.NA
df_clean = df[golden12]
df_clean.to_excel(CLEAN_OUT, index=False)
print(f"Clean Excel saved: {CLEAN_OUT}")

# 5. Markdown report
missing_rate = df_clean.isna().mean().rename("missing_ratio")
report_tbl   = pd.concat([df_clean.describe(include="all").T, missing_rate], axis=1)

md_lines = [
    "# hudaverdi 2010 - cleaning report",
    "",
    f"* source file : `{INPUT_FILE}`",
    f"* original shape : {df_raw.shape}",
    f"* clean shape    : {df_clean.shape}",
    "",
    "## Missing-rate & descriptive stats (Golden-12)",
    report_tbl.to_markdown(),
    "",
]
REPORT_MD.write_text("\n".join(md_lines), encoding="utf-8")
print(f"Markdown report written -> {REPORT_MD}")

print("Finished!")


Reading data/mult_var_blast.xlsx (sheet='hudaverdi_2010')
Original shape : (97, 18)

== dtypes ==
site            object
S/B            float64
H/B            float64
B/D            float64
T/B            float64
PF[kg/m³]      float64
xB[m]          float64
E[Gpa]         float64
x50[m]         float64
FI             float64
group           object
ssB[1/m]       float64
ss50[1/m]      float64
Unnamed: 13    float64
variable        object
OBS            float64
Unnamed: 16    float64
n              float64
dtype: object

== descriptive stats (numeric only) ==
             count       mean        std        min        25%        50%  \
S/B           97.0   1.188969   0.116766   1.000000   1.140000   1.200000   
H/B           97.0   3.345155   1.633681   1.330000   1.830000   2.830000   
B/D           97.0  27.354948   4.838572  17.980000  24.720000  27.270000   
T/B           97.0   1.262784   0.673895   0.500000   0.830000   1.140000   
PF[kg/m³]     97.0   0.529175   0.235455   0.2200

## 4. hudaverdi_2010_full

In [23]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Clean & unify the 'hudaverdi_2010_full' sheet from mult_var_blast.xlsx

"""

# Parameters 
INPUT_FILE = Path("data/mult_var_blast.xlsx")
SHEET_NAME = "hudaverdi_2010_full"

RAW_OUT   = Path("hudaverdi_2010_full_raw.xlsx")
CLEAN_OUT = Path("hudaverdi_2010_full_clean.xlsx")
REPORT_MD = Path("hudaverdi_2010_full_report.md")

# Column mapping: original -> unified 
column_map = {
    "Blast No."     : "blast_id",              
    "Blast ID."     : "blast_code",            
    "S/B"           : "spacing_over_burden",
    "H/B"           : "benchheight_over_burden",
    "B/D"           : "burden_over_diameter",
    "T/B"           : "stemming_over_burden",
    "PF (kg/m3)"    : "powder_factor",
    "XB (m)"        : "x_burden_pred_m",
    "E (GPa)"       : "youngs_modulus_gpa",
    "X50 (m)"       : "fragment_median_m",
    "FI"            : "fragmentation_index",
    # No group / ssB / ss50
}

unit_coef = {}     

golden12 = [
    "site", "blast_id",                  
    "spacing_over_burden",
    "benchheight_over_burden",
    "burden_over_diameter",
    "stemming_over_burden",
    "powder_factor",
    "youngs_modulus_gpa",
    "fragment_median_m",
    "fragmentation_index",
    "group",
    "mesh_area_m2",                      
]

# 1. Load 
print(f"Reading {INPUT_FILE} (sheet='{SHEET_NAME}')")
df_raw = pd.read_excel(INPUT_FILE, sheet_name=SHEET_NAME)
print("Original shape :", df_raw.shape)

# 2. Diagnostics
print("\n== dtypes ==")
print(df_raw.dtypes)
print("\n== descriptive stats (numeric only) ==")
print(df_raw.describe().T)

# 3. Rename & unit convert 
df = df_raw.rename(columns=column_map, errors="ignore")
for col, coef in unit_coef.items():
    if col in df.columns:
        df[col] *= coef

# 4. Save raw & clean 
df_raw.to_excel(RAW_OUT, index=False)
print(f"Raw Excel saved: {RAW_OUT}")

# derive text group from FI 
if "fragmentation_index" in df.columns:
    bins   = [-float("inf"), 4, 8, float("inf")]
    labels = ["FI<4", "4<=FI<=8", "FI>8"]
    df["group"] = pd.cut(
        df["fragmentation_index"],
        bins=bins,
        labels=labels
    )

# guarantee Golden-12 columns
for col in golden12:
    if col not in df.columns:
        df[col] = pd.NA

df_clean = df[golden12]
df_clean.to_excel(CLEAN_OUT, index=False)
print(f"Clean Excel saved: {CLEAN_OUT}")

# 5. Markdown report 
missing_rate = df_clean.isna().mean().rename("missing_ratio")
report_tbl   = pd.concat([df_clean.describe(include="all").T, missing_rate], axis=1)

md_lines = [
    "# hudaverdi 2010 full - cleaning report",
    "",
    f"* source file : `{INPUT_FILE}`",
    f"* original shape : {df_raw.shape}",
    f"* clean shape    : {df_clean.shape}",
    "",
    "## Missing-rate & descriptive stats (Golden-12)",
    report_tbl.to_markdown(),
    "",
]
REPORT_MD.write_text("\n".join(md_lines), encoding="utf-8")
print(f"Markdown report written -> {REPORT_MD}")

print("Finished!")


Reading data/mult_var_blast.xlsx (sheet='hudaverdi_2010_full')
Original shape : (110, 11)

== dtypes ==
Blast No.       int64
Blast ID.      object
S/B           float64
H/B           float64
B/D           float64
T/B           float64
PF (kg/m3)    float64
XB (m)        float64
E (GPa)       float64
X50 (m)       float64
FI            float64
dtype: object

== descriptive stats (numeric only) ==
            count       mean        std    min      25%        50%        75%  \
Blast No.   110.0  55.500000  31.898276   1.00  28.2500  55.500000  82.750000   
S/B         110.0   1.185000   0.114241   1.00   1.1300   1.200000   1.247500   
H/B         110.0   3.338455   1.602141   1.33   2.0700   2.815000   4.687500   
B/D         110.0  27.414909   4.942791  17.98  24.7200  27.270000  30.300000   
T/B         110.0   1.258455   0.672038   0.50   0.8300   1.110000   1.400000   
PF (kg/m3)  110.0   0.539273   0.238742   0.22   0.3525   0.480000   0.682500   
XB (m)      110.0   1.092455   0.

## 5. hudaverdi_2012

In [24]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Clean & unify the 'hudaverdi_2012' sheet from mult_var_blast.xlsx
"""

# Parameters─
INPUT_FILE = Path("data/mult_var_blast.xlsx")
SHEET_NAME = "hudaverdi_2012"

RAW_OUT   = Path("hudaverdi_2012_raw.xlsx")
CLEAN_OUT = Path("hudaverdi_2012_clean.xlsx")
REPORT_MD = Path("hudaverdi_2012_report.md")

# Column mapping: original -> unified
column_map = {
    "blast"        : "blast_id",             
    "b[m]"         : "burden_m",
    "S/B"          : "spacing_over_burden",
    "H/B"          : "benchheight_over_burden",
    "B/D"          : "burden_over_diameter",
    "T/B"          : "stemming_over_burden",
    "PF[kg/m³]"    : "powder_factor",
    "xB[m]"        : "x_burden_pred_m",
    "E[Gpa]"       : "youngs_modulus_gpa",
    "x50[m]"       : "fragment_median_m",
    "FI"           : "fragmentation_index",
    "group"        : "group_numeric",         
    "group_type"   : "group_type",            
}

unit_coef = {}  

golden12 = [
    "site", "blast_id",
    "spacing_over_burden",
    "benchheight_over_burden",
    "burden_over_diameter",
    "stemming_over_burden",
    "powder_factor",
    "youngs_modulus_gpa",
    "fragment_median_m",
    "fragmentation_index",
    "group",              
    "mesh_area_m2",
]

# 1. Load 
print(f"Reading {INPUT_FILE} (sheet='{SHEET_NAME}')")
df_raw = pd.read_excel(INPUT_FILE, sheet_name=SHEET_NAME)
print("Original shape :", df_raw.shape)

# 2. Diagnostics
print("\n== dtypes ==")
print(df_raw.dtypes)
print("\n== descriptive stats (numeric only) ==")
print(df_raw.describe().T)

# 3. Rename & unit convert
df = df_raw.rename(columns=column_map, errors="ignore")
for col, coef in unit_coef.items():
    if col in df.columns:
        df[col] *= coef

# 3-A. Unify group to text label
if "group_type" in df.columns:

    df["group"] = df["group_type"]
elif "fragmentation_index" in df.columns:
    # basing on FI
    bins   = [-float("inf"), 4, 8, float("inf")]
    labels = ["FI<4", "4<=FI<=8", "FI>8"]
    df["group"] = pd.cut(df["fragmentation_index"], bins=bins, labels=labels)

# 4. Save raw & clean 
df_raw.to_excel(RAW_OUT, index=False)
print(f"Raw Excel saved: {RAW_OUT}")

for col in golden12:
    if col not in df.columns:
        df[col] = pd.NA
df_clean = df[golden12]
df_clean.to_excel(CLEAN_OUT, index=False)
print(f"Clean Excel saved: {CLEAN_OUT}")

# 5. Markdown report
missing_rate = df_clean.isna().mean().rename("missing_ratio")
report_tbl   = pd.concat([df_clean.describe(include="all").T, missing_rate], axis=1)

md_lines = [
    "# hudaverdi 2012 - cleaning report",
    "",
    f"* source file : `{INPUT_FILE}`",
    f"* original shape : {df_raw.shape}",
    f"* clean shape    : {df_clean.shape}",
    "",
    "## Missing-rate & descriptive stats (Golden-12)",
    report_tbl.to_markdown(),
    "",
]
REPORT_MD.write_text("\n".join(md_lines), encoding="utf-8")
print(f"Markdown report written -> {REPORT_MD}")

print("Finished!")


Reading data/mult_var_blast.xlsx (sheet='hudaverdi_2012')
Original shape : (62, 16)

== dtypes ==
blast           object
b[m]           float64
S/B            float64
H/B            float64
B/D            float64
T/B            float64
PF[kg/m³]      float64
xB[m]          float64
E[Gpa]         float64
x50[m]         float64
FI             float64
group            int64
group_type      object
Unnamed: 13    float64
variable        object
OBS            float64
dtype: object

== descriptive stats (numeric only) ==
             count       mean       std    min      25%    50%      75%    max
b[m]          62.0   3.249194  2.034850   1.60   2.0000   2.50   2.9250   9.00
S/B           62.0   1.197903  0.133970   1.00   1.1400   1.20   1.2500   1.75
H/B           62.0   3.762258  1.347131   1.67   2.5000   3.50   4.7500   6.82
B/D           62.0  26.680806  5.525936  17.98  22.2825  28.09  32.2525  39.47
T/B           62.0   1.178710  0.385769   0.50   0.8300   1.20   1.5450   1.75
PF[kg/

## 6. hudaverdi_2012_vib

In [25]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Clean & unify the 'hudaverdi_2012_vib' sheet from mult_var_blast.xlsx
"""

# Parameters
INPUT_FILE = Path("data/mult_var_blast.xlsx")
SHEET_NAME = "hudaverdi_2012_vib"

RAW_OUT   = Path("hudaverdi_2012_vib_raw.xlsx")
CLEAN_OUT = Path("hudaverdi_2012_vib_clean.xlsx")
REPORT_MD = Path("hudaverdi_2012_vib_report.md")

# Column mapping: original -> unified
column_map = {
    "blast"         : "blast_id",
    "S/B"           : "spacing_over_burden",
    "H/B"           : "benchheight_over_burden",
    "B/D"           : "burden_over_diameter",
    "T/B"           : "stemming_over_burden",
    "U/B"           : "subdrill_over_burden",
    "PF[kg/m³]"     : "powder_factor",
    "Q/delay[kg]"   : "charge_per_delay_kg",
    "d[m]"          : "distance_m",
    "Sd[kg/Q^0.5]"  : "scaled_distance",
    "PPV[mm/s]"     : "ppv_mms",
    "group"         : "group_numeric",      
}

unit_coef = {}  

golden12 = [
    "site", "blast_id",
    "spacing_over_burden",
    "benchheight_over_burden",
    "burden_over_diameter",
    "stemming_over_burden",
    "powder_factor",
    "youngs_modulus_gpa",      
    "fragment_median_m",       
    "fragmentation_index",     
    "group",                   
    "mesh_area_m2",            
]

# 1. Load
print(f"Reading {INPUT_FILE} (sheet='{SHEET_NAME}')")
df_raw = pd.read_excel(INPUT_FILE, sheet_name=SHEET_NAME)
print("Original shape :", df_raw.shape)

# 2. Diagnostics 
print("\n== dtypes ==")
print(df_raw.dtypes)
print("\n== descriptive stats (numeric only) ==")
print(df_raw.describe().T)

# 3. Rename & unit convert 
df = df_raw.rename(columns=column_map, errors="ignore")
for col, coef in unit_coef.items():
    if col in df.columns:
        df[col] *= coef

# 3-A. numeric group -> text label
if "group_numeric" in df.columns:
    df["group"] = df["group_numeric"].map({1: "FI<4", 2: "4<=FI<=8", 3: "FI>8"}).astype("string")


# 4. Save raw & clean
df_raw.to_excel(RAW_OUT, index=False)
print(f" Raw Excel saved: {RAW_OUT}")

# Golden-12
for col in golden12:
    if col not in df.columns:
        df[col] = pd.NA
df_clean = df[golden12]        

df_vib   = df[["blast_id", "charge_per_delay_kg", "distance_m",
               "scaled_distance", "ppv_mms"]]

df_clean.to_excel(CLEAN_OUT, index=False)
print(f" Clean Excel saved: {CLEAN_OUT}")

# 5. Markdown report
missing_rate = df_clean.isna().mean().rename("missing_ratio")
report_tbl   = pd.concat([df_clean.describe(include="all").T, missing_rate],
                         axis=1)

md_lines = [
    "# hudaverdi 2012 vib - cleaning report",
    "",
    f"* source file : `{INPUT_FILE}`",
    f"* original shape : {df_raw.shape}",
    f"* clean shape    : {df_clean.shape}",
    "",
    "## Missing-rate & descriptive stats (Golden-12)",
    report_tbl.to_markdown(),
    "",
    "## Extra vibration columns kept (not in Golden-12)",
    "`charge_per_delay_kg`, `distance_m`, `scaled_distance`, `ppv_mms`",
]
REPORT_MD.write_text("\n".join(md_lines), encoding="utf-8")
print(f"Markdown report written -> {REPORT_MD}")

print("Finished!")


Reading data/mult_var_blast.xlsx (sheet='hudaverdi_2012_vib')
Original shape : (88, 21)

== dtypes ==
blast             int64
S/B             float64
H/B             float64
B/D             float64
T/B             float64
U/B             float64
PF[kg/m³]       float64
Q/delay[kg]     float64
d[m]              int64
Sd[kg/Q^0.5]    float64
PPV[mm/s]       float64
group             int64
Unnamed: 12     float64
variable         object
OBS             float64
Unnamed: 15     float64
variable.1       object
min             float64
max             float64
mean            float64
std_dev         float64
dtype: object

== descriptive stats (numeric only) ==
              count        mean        std     min        25%       50%  \
blast          88.0   44.500000  25.547342   1.000   22.75000   44.5000   
S/B            88.0    1.188409   0.058700   1.000    1.14000    1.2000   
H/B            88.0    3.215568   0.487624   2.400    2.89000    3.1800   
B/D            88.0   25.178295   2.5826

## 7. trivedi_2014_flyrock

In [26]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Clean & unify the 'trivedi_2014_flyrock' sheet from mult_var_blast.xlsx
"""

# Parameters 
INPUT_FILE = Path("data/mult_var_blast.xlsx")
SHEET_NAME = "trivedi_2014_flyrock"
RAW_OUT   = Path("trivedi_2014_flyrock_raw.xlsx")
CLEAN_OUT = Path("trivedi_2014_flyrock_clean.xlsx")
REPORT_MD = Path("trivedi_2014_flyrock_report.md")

# Column mapping: original -> unified
column_map = {
    "Blast No."       : "blast_id",
    "Name of the mine": "site",
    "d(mm)"           : "hole_diameter_mm",
    "Q(kg)"           : "charge_per_hole_kg",
    "ql(kg/m)"        : "linear_charge_density",
    "lb(m)"           : "hole_depth_m",
    "B(m)"            : "burden_m",
    "Sb(m)"           : "spacing_m",
    "ls(m)"           : "stemming_m",
    "q(kg/t)"         : "specific_charge_kgt",
    "σc(MPa)"         : "ucs_mpa",
    "RQD(%)"          : "rqd_pct",
    "v0(m/s)"         : "flyrock_velocity_ms",
    "θ0(°)"           : "flyrock_angle_deg",
    "Rf(m)"           : "flyrock_distance_m",
}

unit_coef = {
    "hole_diameter_mm": 0.001,        # mm -> m
}

# Golden-12 
golden12 = [
    "site", "blast_id",
    "spacing_over_burden",
    "benchheight_over_burden",        # None -> NaN
    "burden_over_diameter",
    "stemming_over_burden",
    "powder_factor",                  # None -> NaN
    "youngs_modulus_gpa",             # None -> NaN
    "fragment_median_m",              # None -> NaN
    "fragmentation_index",            # None -> NaN
    "group",                          # None -> NaN
    "mesh_area_m2",
]

# 1. Load
print(f"Reading {INPUT_FILE} (sheet='{SHEET_NAME}')")
df_raw = pd.read_excel(INPUT_FILE, sheet_name=SHEET_NAME)
print("Original shape :", df_raw.shape)

# 2. Diagnostics
df = df_raw.rename(columns=column_map, errors="ignore")
for col, coef in unit_coef.items():
    if col in df.columns:
        df[col] = df[col] * coef

# 3. Rename & unit convert
# 3-A mesh area
if {"spacing_m", "burden_m"}.issubset(df.columns):
    df["mesh_area_m2"] = df["spacing_m"] * df["burden_m"]
    df["spacing_over_burden"] = df["spacing_m"] / df["burden_m"]

# 3-B burden_over_diameter
if {"burden_m", "hole_diameter_mm"}.issubset(df.columns):
    df["burden_over_diameter"] = df["burden_m"] / df["hole_diameter_mm"]

# 3-C stemming_over_burden
if {"stemming_m", "burden_m"}.issubset(df.columns):
    df["stemming_over_burden"] = df["stemming_m"] / df["burden_m"]

# 3-D fill other Golden-12 columns with NA
for col in golden12:
    if col not in df.columns:
        df[col] = pd.NA

# 4. Save raw & clean
df_raw.to_excel(RAW_OUT, index=False)
print(f" Raw Excel saved: {RAW_OUT}")

df_clean = df[golden12]
df_clean.to_excel(CLEAN_OUT, index=False)
print(f" Clean Excel saved: {CLEAN_OUT}")

# 5. Markdown report
missing_rate = df_clean.isna().mean().rename("missing_ratio")
report_tbl   = pd.concat([df_clean.describe(include="all").T, missing_rate], axis=1)

md_lines = [
    "# trivedi 2014 flyrock  cleaning report",
    "",
    f"* source file : `{INPUT_FILE}`",
    f"* original shape : {df_raw.shape}",
    f"* clean shape    : {df_clean.shape}",
    "",
    "## Missing-rate & descriptive stats (Golden-12)",
    report_tbl.to_markdown(),
    "",
    "## Extra fly-rock features kept",
    "`charge_per_hole_kg`, `linear_charge_density`, `hole_depth_m`, "
    "`specific_charge_kgt`, `ucs_mpa`, `rqd_pct`, "
    "`flyrock_velocity_ms`, `flyrock_angle_deg`, `flyrock_distance_m`",
]
REPORT_MD.write_text("\n".join(md_lines), encoding="utf-8")
print(f"Markdown report written -> {REPORT_MD}")

print("Finished!")


Reading data/mult_var_blast.xlsx (sheet='trivedi_2014_flyrock')
Original shape : (30, 19)
 Raw Excel saved: trivedi_2014_flyrock_raw.xlsx
 Clean Excel saved: trivedi_2014_flyrock_clean.xlsx
Markdown report written -> trivedi_2014_flyrock_report.md
Finished!


## 8. sharma_2017

In [27]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Clean & unify the 'sharma_2017' sheet from mult_var_blast.xlsx
"""

# Parameters
INPUT_FILE = Path("data/mult_var_blast.xlsx")
SHEET_NAME = "sharma_2017"
RAW_OUT   = Path("sharma_2017_raw.xlsx")
CLEAN_OUT = Path("sharma_2017_clean.xlsx")
REPORT_MD = Path("sharma_2017_report.md")

# Column mapping: original -> unified
column_map = {
    "blast_ID" : "blast_id",
    # Geometric parameters
    "D" : "hole_diameter_m",
    "H" : "bench_height_m",
    "J" : "subgrade_drilling_m",
    "S" : "spacing_m",
    "B" : "burden_m",
    "T" : "stemming_m",
    "L" : "length_m",
    "W" : "width_m",
    # ratio (given)
    "S/B" : "spacing_over_burden",
    "T/B" : "stemming_over_burden",
    "H/B" : "benchheight_over_burden",
    "J/B" : "subgrade_over_burden",
    "B/D" : "burden_over_diameter",
    "L/W" : "length_over_width",
    # Blasting parameters
    "NH" : "num_holes",
    "Qe" : "total_explosive_t",
    "De" : "linear_explosive_density",
    "PF" : "powder_factor_m3kg",   # m3/kg
    # Rock & results
    "UCS" : "ucs_mpa",
    "MFS" : "mean_fragment_size_m",
    "dataset" : "dataset_id",
}

unit_coef = {

}

golden12 = [
    "site", "blast_id",
    "spacing_over_burden",
    "benchheight_over_burden",
    "burden_over_diameter",
    "stemming_over_burden",
    "powder_factor",
    "youngs_modulus_gpa",
    "fragment_median_m",
    "fragmentation_index",
    "group",
    "mesh_area_m2",
]

# 1. Load
print(f"Reading {INPUT_FILE} (sheet='{SHEET_NAME}')")
df_raw = pd.read_excel(INPUT_FILE, sheet_name=SHEET_NAME)
print("Original shape :", df_raw.shape)

# 2. Diagnostics
df = df_raw.rename(columns=column_map, errors="ignore")

# 3. Rename & unit convert
# powder factor m3/kg -> kg/m3
if "powder_factor_m3kg" in df.columns:
    pf = df["powder_factor_m3kg"]
    df["powder_factor"] = np.where(pf > 0, 1.0 / pf, pd.NA)

# 4. Supplement columns
# 4-A mesh area
if {"spacing_m", "burden_m"}.issubset(df.columns):
    df["mesh_area_m2"] = df["spacing_m"] * df["burden_m"]

# 4-B fragment_median_m use MFS
if "mean_fragment_size_m" in df.columns:
    df["fragment_median_m"] = df["mean_fragment_size_m"]

# 5. Save
df_raw.to_excel(RAW_OUT, index=False)
print(f" Raw Excel saved: {RAW_OUT}")

# Golden-12
for col in golden12:
    if col not in df.columns:
        df[col] = pd.NA
df_clean = df[golden12]
df_clean.to_excel(CLEAN_OUT, index=False)
print(f"Clean Excel saved: {CLEAN_OUT}")

# 6. Markdown report
missing_rate = df_clean.isna().mean().rename("missing_ratio")
report_tbl   = pd.concat([df_clean.describe(include="all").T, missing_rate], axis=1)

md_lines = [
    "# sharma 2017 - cleaning report",
    "",
    f"* source file : `{INPUT_FILE}`",
    f"* original shape : {df_raw.shape}",
    f"* clean shape    : {df_clean.shape}",
    "",
    "## Missing-rate & descriptive stats (Golden-12)",
    report_tbl.to_markdown(),
    "",
    "# Extra columns kept (not in Golden-12)",
    "`hole_diameter_m`, `bench_height_m`, `subgrade_drilling_m`, `spacing_m`, "
    "`burden_m`, `stemming_m`, `length_m`, `width_m`, "
    "`subgrade_over_burden`, `length_over_width`, "
    "`num_holes`, `total_explosive_t`, `linear_explosive_density`, "
    "`ucs_mpa`, `dataset_id`",
]
REPORT_MD.write_text("\n".join(md_lines), encoding="utf-8")
print(f"Markdown report written -> {REPORT_MD}")

print("Finished!")

Reading data/mult_var_blast.xlsx (sheet='sharma_2017')
Original shape : (76, 50)
 Raw Excel saved: sharma_2017_raw.xlsx
Clean Excel saved: sharma_2017_clean.xlsx
Markdown report written -> sharma_2017_report.md
Finished!


# Data Merging

In [13]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Merge three FI-oriented clean tables into a single Excel file.
This is a light-weight step for data-scarce settings.
"""

# Input files (all have the same Golden-12 columns, plus extras)
INPUT_FILES = [
    Path("clean_data/hudaverdi_2010_full_clean.xlsx"),
    Path("clean_data/hudaverdi_2012_clean.xlsx"),
    Path("clean_data/kulatilake_2010_clean.xlsx"),
]

# Output paths
OUT_XLSX   = Path("clean_data/fi_pool.xlsx")
OUT_REPORT = Path("clean_data/fi_pool_summary.md")

# 1. Read each file and tag its source
frames = []
for fp in INPUT_FILES:
    df = pd.read_excel(fp, engine="openpyxl")
    df["dataset"] = fp.stem.replace("_clean", "")   # e.g., hudaverdi_2012
    frames.append(df)

# 2. Vertical concatenation (columns auto-aligned, NaN where missing)
fi_pool = pd.concat(frames, ignore_index=True, sort=False)

# Build a unique row index in case different sets share blast IDs
fi_pool["uid"] = fi_pool["dataset"].astype(str) + "_" + fi_pool["blast_id"].astype(str)
fi_pool = fi_pool.set_index("uid", drop=True)

# 3. Save to Excel
OUT_XLSX.parent.mkdir(parents=True, exist_ok=True)
fi_pool.to_excel(OUT_XLSX, index=True, engine="openpyxl")
print(f"Saved merged file: {OUT_XLSX} ({fi_pool.shape[0]} rows, {fi_pool.shape[1]} columns)")



Saved merged file: clean_data/fi_pool.xlsx (262 rows, 13 columns)
Summary written: clean_data/fi_pool_summary.md


In [19]:
df = pd.read_excel("data/merged_data/fi_pool.xlsx", engine="openpyxl")

# 1. Drop the empty column
df = df.drop(columns=["mesh_area_m2"], errors="ignore")

# 2. Set uid as index
df = df.set_index("uid", drop=True)

# 3.  drop meta columns
df = df.drop(columns=["site", "blast_id"])

# 4. Save back to Excel
df.to_excel("data/merged_data/fi_pool.xlsx", engine="openpyxl")
print("Trimmed dataset saved.")

Trimmed dataset saved.


# Ploting and Statistical Comparison 

In [3]:
"""
Compute stats and generate figures for the FI pool.
Inputs:
  - data/merged_data/fi_pool.xlsx        
Outputs:
  - outputs/stats_missing_by_source.csv
  - outputs/stats_summary_by_source.csv
  - outputs/fig_D1_ecdf.png
  - outputs/fig_D2_corr.png
  - outputs/fig_D2_delta_corr.png
"""

# I/O paths
PREF = Path("data/merged_data/fi_pool.xlsx")
OUTD = Path("figure"); OUTD.mkdir(parents=True, exist_ok=True)

# Load data
if PREF.exists():
    df = pd.read_excel(PREF, engine="openpyxl")

# make sure index and dataset
if "uid" in df.columns:
    df = df.set_index("uid", drop=True)
if "dataset" not in df.columns:
    raise ValueError("`dataset` column is required to group by source.")

# Optionally drop clearly meta columns if present
drop_cols = [c for c in ["site","blast_id","mesh_area_m2"] if c in df.columns]
df = df.drop(columns=drop_cols)

# Feature lists

# core features used in figures
feat_core = [
    "fragmentation_index",        # FI
    "fragment_median_m",          # X50
    "powder_factor",              # PF (kg/m^3)
    "benchheight_over_burden",    # H/B
    "burden_over_diameter",       # B/D
    "stemming_over_burden",       # T/B
]
# optional extras for correlation
feat_corr = [f for f in (feat_core + [
    "spacing_over_burden", "youngs_modulus_gpa"
]) if f in df.columns]


# Missingness & summary stats

# missing by feature & source
miss = (df.groupby("dataset")[feat_core]
          .apply(lambda x: x.isna().mean()*100.0)
          .round(1))
miss.to_csv(OUTD/"stats_missing_by_source.csv")

# numeric summary by source
def summary_tbl(g):
    s = g[feat_core].agg(["count","mean","std","min","median","max"]).T
    s["skew"] = g[feat_core].skew(numeric_only=True)
    return s

summ = []
for name, g in df.groupby("dataset"):
    t = summary_tbl(g)
    t.insert(0, "dataset", name)
    summ.append(t.reset_index().rename(columns={"index":"feature"}))
summ = pd.concat(summ, ignore_index=True)
summ.to_csv(OUTD/"stats_summary_by_source.csv", index=False)

print(f"[OK] Missingness saved -> {OUTD/'stats_missing_by_source.csv'}")
print(f"[OK] Summary saved     -> {OUTD/'stats_summary_by_source.csv'}")


# 4) Figure D1: ECDF by source (6 panels in one figure)
def ecdf(y):
    y = np.asarray(y)
    y = y[~np.isnan(y)]
    if y.size == 0: 
        return np.array([]), np.array([])
    y_sorted = np.sort(y)
    x = y_sorted
    F = np.arange(1, y_sorted.size+1) / y_sorted.size
    return x, F

panels = [f for f in feat_core if f in df.columns]
n_pan = len(panels)
ncols  = 3
nrows  = int(np.ceil(n_pan/ncols))

fig = plt.figure(figsize=(12, 3.6*nrows), dpi=200)
plt.suptitle("Per-source ECDFs for key features", y=0.995)

for i, f in enumerate(panels, 1):
    ax = fig.add_subplot(nrows, ncols, i)
    for src, g in df.groupby("dataset"):
        x, F = ecdf(g[f].values)
        if x.size == 0: 
            continue
        # log-like axis for PF / X50 when all positive
        if f in ["fragment_median_m","powder_factor"] and np.all(x>0):
            ax.set_xscale("log")
        ax.step(x, F, where="post", label=str(src), alpha=0.9)
    ax.set_xlabel(f)
    ax.set_ylabel("ECDF")
    ax.grid(True, alpha=0.3)
    if i == 1:
        ax.legend(loc="lower right", fontsize=8)

fig.tight_layout(rect=[0,0,1,0.98])
fig.savefig(OUTD/"fig_D1_ecdf.png")
plt.close(fig)
print(f"[OK] Figure D1 saved   -> {OUTD/'fig_D1_ecdf.png'}")


# Figure D2: Correlation heatmaps (pooled + per source)
def corr_mat(frame, cols):
    # Spearman is robust to tail/heavy-tailed marginals
    return frame[cols].corr(method="spearman")

C_pooled = corr_mat(df, feat_corr)

# layout: 2x2 (pooled + first 2 sources), others appended if exist
sources = list(df["dataset"].unique())
# limit to at most 3 sources for a compact 2x2 grid; if 3, we do pooled + 3 sources in 2x2
sources = sources[:min(3, len(sources))]

def heat(ax, M, title):
    im = ax.imshow(M, vmin=-1, vmax=1, interpolation="nearest")
    ax.set_xticks(range(M.shape[1])); ax.set_xticklabels(M.columns, rotation=90)
    ax.set_yticks(range(M.shape[0])); ax.set_yticklabels(M.index)
    ax.set_title(title, fontsize=11)
    for (i,j), v in np.ndenumerate(M.values):
        if not np.isnan(v):
            ax.text(j, i, f"{v:.2f}", ha="center", va="center", fontsize=7)
    ax.grid(False)
    return im

# main heatmaps
n_sub = 1 + len(sources)
ncols = 2
nrows = int(np.ceil(n_sub/ncols))
fig2, axes = plt.subplots(nrows, ncols, figsize=(5.5*ncols, 5.5*nrows), dpi=200)
axes = np.atleast_1d(axes).ravel()

im = heat(axes[0], C_pooled, "Pooled (Spearman)")
for k, src in enumerate(sources, start=1):
    C_src = corr_mat(df[df["dataset"]==src], feat_corr)
    heat(axes[k], C_src, f"{src} (Spearman)")

# colorbar
for ax in axes[n_sub:]:
    ax.axis("off")
cax = fig2.add_axes([0.92, 0.15, 0.02, 0.7])
cb = fig2.colorbar(axes[0].images[0], cax=cax)
cb.set_label("Correlation")

fig2.tight_layout(rect=[0,0,0.9,1])
fig2.savefig(OUTD/"fig_D2_corr.png")
plt.close(fig2)
print(f"[OK] Figure D2 saved   -> {OUTD/'fig_D2_corr.png'}")

# Delta-correlation heatmaps (per-source minus pooled) — optional but useful
fig3, axes3 = plt.subplots(len(sources), 1, figsize=(6.2, 4.8*len(sources)), dpi=200)
if len(sources)==1:
    axes3 = [axes3]
for ax, src in zip(axes3, sources):
    C_src = corr_mat(df[df["dataset"]==src], feat_corr)
    D = (C_src - C_pooled).fillna(0)
    im = ax.imshow(D, vmin=-1, vmax=1)
    ax.set_xticks(range(D.shape[1])); ax.set_xticklabels(D.columns, rotation=90)
    ax.set_yticks(range(D.shape[0])); ax.set_yticklabels(D.index)
    ax.set_title(f"delta-Correlation ( {src} - pooled )", fontsize=11)
    for (i,j), v in np.ndenumerate(D.values):
        ax.text(j, i, f"{v:+.2f}", ha="center", va="center", fontsize=7)
cbar = fig3.colorbar(im, ax=axes3, fraction=0.015, pad=0.02)
cbar.set_label("delta Spearman")
fig3.tight_layout()
fig3.savefig(OUTD/"fig_D2_delta_corr.png")
plt.close(fig3)
print(f"[OK] Figure D2 saved  -> {OUTD/'fig_2_delta_corr.png'}")


[OK] Missingness saved -> figure/stats_missing_by_source.csv
[OK] Summary saved     -> figure/stats_summary_by_source.csv
[OK] Figure D1 saved   -> figure/fig_D1_ecdf.png


/var/folders/3g/h5_sbcb90jnd465r4rgty63w0000gp/T/ipykernel_6783/3635503685.py:157: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig2.tight_layout(rect=[0,0,0.9,1])


[OK] Figure D2 saved   -> figure/fig_D2_corr.png


/var/folders/3g/h5_sbcb90jnd465r4rgty63w0000gp/T/ipykernel_6783/3635503685.py:177: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig3.tight_layout()


[OK] Figure D2 saved  -> figure/fig_2_delta_corr.png
